In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
pip install torchvision

In [ ]:
from torchvision import models
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import time
import copy
import os

In [ ]:
shufflenet = models.shufflenet_v2_x1_0(pretrained=True)
for name,child in shufflenet.named_children():
  print(name)

conv1
maxpool
stage2
stage3
stage4
conv5
fc


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
def save_checkpoint(state, filename="/content/drive/MyDrive/Checkpoints/phase3_shufflenet_transparent_trial_3_dropout.pth"):
  print("Saving checkpoint")
  torch.save(state, filename)

In [ ]:
for name, child in shufflenet.named_children():
   if name in ['stage3','stage4','conv5','fc']:
       print(name + ' is unfrozen')
       for param in child.parameters():
           param.requires_grad = True
   elif name in ['conv1','maxpool','stage2']:
       print(name + ' is frozen')
       for param in child.parameters():
           param.requires_grad = False

shufflenet.last_linear = nn.Sequential(
    nn.ReLU(),  
    nn.Dropout(0.20),
    nn.Linear(512, 128),
    nn.ReLU(),  
    nn.Dropout(0.25), 
    nn.Linear(in_features=128, out_features=2))

shufflenet = shufflenet.cuda() if device else shufflenet

conv1 is frozen
maxpool is frozen
stage2 is frozen
stage3 is unfrozen
stage4 is unfrozen
conv5 is unfrozen
fc is unfrozen


In [ ]:
from torchvision import transforms
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
image_transforms = transforms.Compose([
                            transforms.Resize((224,224)),
                            transforms.ColorJitter(hue=.05, saturation=.05),
                            transforms.RandomRotation(degrees=15),
                            transforms.RandomHorizontalFlip(),
                            transforms.ToTensor(),
                            normalize,
                        ])

In [ ]:
from torchvision import *
from torch.utils.data import Dataset, DataLoader
# Load the Data

# Set train and valid directory paths
train_directory = '/content/drive/MyDrive/img final dset/Train'
valid_directory = '/content/drive/MyDrive/img final dset/Test'

# Batch size
bs = 32

# Load Data from folders
data = {
    'train': datasets.ImageFolder(train_directory, image_transforms),
    'test': datasets.ImageFolder(valid_directory, image_transforms)
}

# Size of Data, to be used for calculating Average Loss and Accuracy
train_data_size = len(data['train'])
test_data_size = len(data['test'])

# Create iterators for the Data loaded using DataLoader module
train_data = DataLoader(data['train'], batch_size=bs, shuffle=True)
test_data = DataLoader(data['test'], batch_size=bs, shuffle=True)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Print the train, validation and test set data sizes
train_data_size, test_data_size

(27648, 6912)

In [ ]:
# HYPER-PARAMETERS
epochs=60
train_only_last_layer = False # boolean variable
num_of_output_classes=2

In [ ]:
# Learning rate scheduler
# ^^^^^^^^^^^^^^^^^^^^^^^
# Let's create our learning rate scheduler. We will exponentially
# decrease the learning rate once every few epochs.

"""This function is useful only if using SGD otherwise no use"""

def exp_lr_scheduler(optimizer, epoch, init_lr=0.001, lr_decay_epoch=7):
    """Decay learning rate by a factor of 0.1 every lr_decay_epoch epochs."""
    lr = init_lr * (0.1**(epoch // lr_decay_epoch))

    if epoch % lr_decay_epoch == 0:
        print('LR is set to {}'.format(lr))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return optimizer

In [ ]:
# defining loss criterion, for these
# models CrossEntropyLoss works the best
criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opoosed to before.
""" Defining an optimiser function here, can use Adam, RMSprop or simple SGD"""
optimizer_conv = optim.SGD(shufflenet.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-5, nesterov=True)

In [ ]:
# Training the model
# ------------------
#
# -  Saving (deep copying) the best model
#
# In the following, parameter ``lr_scheduler(optimizer, epoch)``
# is a function  which modifies ``optimizer`` so that the learning
# rate is changed according to desired schedule.

def train_model(model, criterion, optimizer, lr_scheduler, num_epochs):

  for epoch in range(epochs):
     epoch_start = time.time()
     print("Epoch: {}/{}".format(epoch+1, epochs))

     if epoch==1 or epoch==20 or epoch==45:
       checkpoint= {'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
       save_checkpoint(checkpoint)
     
     # Set to training mode
     shufflenet.train()
     
     # Loss and Accuracy within the epoch
     train_loss = 0.0
     train_acc = 0.0
     
     valid_loss = 0.0
     valid_acc = 0.0
     
     for i, (inputs, labels) in enumerate(train_data):
       
       inputs = inputs.to(device)
       labels = labels.to(device)
       
       # Clean existing gradients
       optimizer_conv.zero_grad()
       
       # Forward pass - compute outputs on input data using the model
       outputs = shufflenet(inputs)
       
       # Compute loss
       loss = criterion(outputs, labels)
       
       # Backpropagate the gradients
       loss.backward()
       
       # Update the parameters
       optimizer_conv.step()
       
       # Compute the total loss for the batch and add it to train_loss
       train_loss += loss.item() * inputs.size(0)
       
       # Compute the accuracy
       ret, predictions = torch.max(outputs.data, 1)
       correct_counts = predictions.eq(labels.data.view_as(predictions))
       
       # Convert correct_counts to float and then compute the mean
       acc = torch.mean(correct_counts.type(torch.FloatTensor))
       
       # Compute total accuracy in the whole batch and add to train_acc
       train_acc += acc.item() * inputs.size(0)
       print("Batch number: {:03d}, Training:  Loss: {:.4f}, Accuracy: {:.4f}".format(i, loss.item(), acc.item()))

In [ ]:
train_model(shufflenet,criterion,optimizer_conv,exp_lr_scheduler,epochs)

Streaming output truncated to the last 5000 lines.
Batch number: 754, Training:  Loss: 0.0005, Accuracy: 1.0000
Batch number: 755, Training:  Loss: 0.0146, Accuracy: 1.0000
Batch number: 756, Training:  Loss: 0.0027, Accuracy: 1.0000
Batch number: 757, Training:  Loss: 0.0017, Accuracy: 1.0000
Batch number: 758, Training:  Loss: 0.0036, Accuracy: 1.0000
Batch number: 759, Training:  Loss: 0.0030, Accuracy: 1.0000
Batch number: 760, Training:  Loss: 0.0031, Accuracy: 1.0000
Batch number: 761, Training:  Loss: 0.0965, Accuracy: 0.9688
Batch number: 762, Training:  Loss: 0.0016, Accuracy: 1.0000
Batch number: 763, Training:  Loss: 0.0057, Accuracy: 1.0000
Batch number: 764, Training:  Loss: 0.0014, Accuracy: 1.0000
Batch number: 765, Training:  Loss: 0.0009, Accuracy: 1.0000
Batch number: 766, Training:  Loss: 0.0028, Accuracy: 1.0000
Batch number: 767, Training:  Loss: 0.0007, Accuracy: 1.0000
Batch number: 768, Training:  Loss: 0.0010, Accuracy: 1.0000
Batch number: 769, Training:  Loss

In [ ]:
def computeTestSetAccuracy(model, loss_criterion):
    '''
    Function to compute the accuracy on the test set
    Parameters
        :param model: Model to test
        :param loss_criterion: Loss Criterion to minimize
    '''

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    test_acc = 0.0
    test_loss = 0.0

    # Validation - No gradient tracking needed
    with torch.no_grad():

        # Set to evaluation mode
        model.eval()

        # Validation loop
        for j, (inputs, labels) in enumerate(test_data):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)

            # Compute loss
            loss = loss_criterion(outputs, labels)

            # Compute the total loss for the batch and add it to valid_loss
            test_loss += loss.item() * inputs.size(0)

            # Calculate validation accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))

            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))

            # Compute total accuracy in the whole batch and add to valid_acc
            test_acc += acc.item() * inputs.size(0)

            print("Test Batch number: {:03d}, Test: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))

    # Find average test loss and test accuracy
    avg_test_loss = test_loss/test_data_size 
    avg_test_acc = test_acc/test_data_size

    print("Test accuracy : " + str(avg_test_acc))

In [ ]:
computeTestSetAccuracy(shufflenet,criterion)

Test Batch number: 000, Test: Loss: 5.9853, Accuracy: 0.5938
Test Batch number: 001, Test: Loss: 7.1340, Accuracy: 0.5312
Test Batch number: 002, Test: Loss: 9.3866, Accuracy: 0.4375
Test Batch number: 003, Test: Loss: 9.5402, Accuracy: 0.4375
Test Batch number: 004, Test: Loss: 8.3127, Accuracy: 0.4688
Test Batch number: 005, Test: Loss: 6.2054, Accuracy: 0.5938
Test Batch number: 006, Test: Loss: 7.4648, Accuracy: 0.5000
Test Batch number: 007, Test: Loss: 7.1688, Accuracy: 0.5312
Test Batch number: 008, Test: Loss: 6.2773, Accuracy: 0.5938
Test Batch number: 009, Test: Loss: 6.9335, Accuracy: 0.5000
Test Batch number: 010, Test: Loss: 7.5362, Accuracy: 0.5000
Test Batch number: 011, Test: Loss: 8.3816, Accuracy: 0.5312
Test Batch number: 012, Test: Loss: 8.8667, Accuracy: 0.4062
Test Batch number: 013, Test: Loss: 8.5604, Accuracy: 0.4688
Test Batch number: 014, Test: Loss: 7.4907, Accuracy: 0.5000
Test Batch number: 015, Test: Loss: 6.4543, Accuracy: 0.5312
Test Batch number: 016, 